#Importation des bibliothéques nécessaires

In [2]:
import pandas as pd 
from google.colab import drive
import re
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [3]:
# Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
import keras
from keras.optimizers import SGD, Adam
from keras.layers import Embedding

In [4]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#Étape 1 : Chargement et prétraitement des données

In [6]:
drive.mount('/content/drive',force_remount=False)

df = pd.read_csv("/content/drive/MyDrive/french_tweets.csv",
                    header=None,encoding_errors='ignore') 


Mounted at /content/drive


<ipython-input-6-54eb4dbb0294>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/french_tweets.csv",


In [7]:
df.head() #on voit que le nom des colonnes sont sur la première ligne, on va donc la supprimer et renommer les colonnes

,0,1
0,label,text
1,0,"- Awww, c'est un bummer. Tu devrais avoir davi..."
2,0,Est contrarié qu'il ne puisse pas mettre à jou...
3,0,J'ai plongé plusieurs fois pour la balle. A ré...
4,0,Tout mon corps a des démangeaisons et comme si...


In [8]:
#supprime la premiere ligne
df = df.drop(0,axis =0)

In [9]:
# Renommage des colonnes et sélection des colonnes utiles
df.columns = ["target", "tweets"]
df = df[["target", "tweets"]]

In [10]:
df.head()

,target,tweets
1,0,"- Awww, c'est un bummer. Tu devrais avoir davi..."
2,0,Est contrarié qu'il ne puisse pas mettre à jou...
3,0,J'ai plongé plusieurs fois pour la balle. A ré...
4,0,Tout mon corps a des démangeaisons et comme si...
5,0,"Non, il ne se comporte pas du tout. je suis en..."


In [11]:
#On choisit 10000 d'éléments pour la phase d'apprentissage
N=10000
df = pd.concat([df.head(N),df.tail(N)], axis=0)

#Étape 2 : Préparation des données pour le modèle


In [12]:
#nettoyage des données 
stop_words = stopwords.words('french')

def nettoyage(text):
    text = text.lower()
    text = text.replace('\n', ' ').replace('\r', '')
    text = ' '.join(text.split())
    text = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", text)
    text = re.sub(r"(\s\-\s|-$)", "", text)
    text = re.sub(r"[,\!\?\%\(\)\/\"]", "", text)
    text = re.sub(r"\&\S*\s", "", text)
    text = re.sub(r"\&", "", text)
    text = re.sub(r"\+", "", text)
    text = re.sub(r"\#", "", text)
    text = re.sub(r"\$", "", text)
    text = re.sub(r"\£", "", text)
    text = re.sub(r"\%", "", text)
    text = re.sub(r"\:", "", text)
    text = re.sub(r"\@", "", text)
    text = re.sub(r"\-", "", text)
    words = [word for word in text if word not in stop_words]
    return text


In [13]:
df["tweets"] = df["tweets"].apply(nettoyage)

In [14]:
train_data,test_data = train_test_split(df, test_size=0.2, random_state=42)

In [15]:
train_tweets = train_data['tweets']
train_labels = train_data['target']
test_tweets = test_data['tweets']
test_labels = test_data['target']

In [16]:
# Initialisez un Tokenizer avec un nombre maximum de mots de 5000
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_tweets)

In [17]:
# Convertir les tweets en séquences de nombres (indices de mots)
train_tweets = tokenizer.texts_to_sequences(train_tweets)
test_tweets = tokenizer.texts_to_sequences(test_tweets)

In [18]:
# Remplir (padding) les séquences pour qu'elles aient toutes la même longueur
maxlen = 50
train_sequences = pad_sequences(train_tweets, padding='post', maxlen=maxlen)
test_sequences = pad_sequences(test_tweets, padding='post', maxlen=maxlen)

In [19]:
train_sequences

array([[  17,  424,   45, ...,    0,    0,    0],
       [ 857,   17,   86, ...,    0,    0,    0],
       [   2,  966, 2052, ...,    0,    0,    0],
       ...,
       [  55,   16,  130, ...,    0,    0,    0],
       [  12,   21, 1129, ...,    0,    0,    0],
       [  42,  297, 2024, ...,    0,    0,    0]], dtype=int32)

In [20]:
# Convertir les labels en vecteurs binaires
train_labels = train_labels.astype('int')
test_labels = test_labels.astype('int')
train_labels = np.eye(2)[train_labels]
test_labels = np.eye(2)[test_labels]

In [21]:
train_labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

#Étape 3 : Développement et entraînement du modèle

In [22]:
# Définir le modèle LSTM
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=50, input_length=maxlen))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=2, activation='softmax'))

In [23]:
# Compiler le modèle
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [27]:
#Entraîner le modèle
history = model.fit(train_sequences, train_labels, epochs=10, batch_size=10000, validation_split=0.2) 
#augmenter le batch size plus grand peut aller plus vite mais on risque le surapprentissage , on peut aussi changer la fonction de perte ou d'optimizer

Epoch 1/10
2/2 [==============================] - 16s 4s/step - loss: 0.4944 - accuracy: 0.7823 - val_loss: 0.5544 - val_accuracy: 0.7312
Epoch 2/10
2/2 [==============================] - 16s 4s/step - loss: 0.4708 - accuracy: 0.8004 - val_loss: 0.5789 - val_accuracy: 0.7150
Epoch 3/10
2/2 [==============================] - 16s 4s/step - loss: 0.4586 - accuracy: 0.8024 - val_loss: 0.5813 - val_accuracy: 0.7375
Epoch 4/10
2/2 [==============================] - 16s 4s/step - loss: 0.4445 - accuracy: 0.8161 - val_loss: 0.5423 - val_accuracy: 0.7337
Epoch 5/10
2/2 [==============================] - 16s 4s/step - loss: 0.4292 - accuracy: 0.8286 - val_loss: 0.5847 - val_accuracy: 0.7359
Epoch 6/10
2/2 [==============================] - 18s 6s/step - loss: 0.4112 - accuracy: 0.8348 - val_loss: 0.5791 - val_accuracy: 0.7341
Epoch 7/10
2/2 [==============================] - 16s 5s/step - loss: 0.3984 - accuracy: 0.8414 - val_loss: 0.5613 - val_accuracy: 0.7303
Epoch 8/10
2/2 [==================

#Étape 4 : Évaluation du modèle

In [28]:
perte, précision = model.evaluate(test_sequences,test_labels)
print('Précision :', précision)
print('Perte:', perte)

125/125 [==============================] - 4s 28ms/step - loss: 0.6614 - accuracy: 0.7143
Précision : 0.7142500281333923
Perte: 0.6614165306091309


In [29]:
# Prédiction sur les données de test
y_pred = model.predict(test_sequences)
y_pred = (y_pred > 0.5).astype('int')

125/125 [==============================] - 4s 28ms/step


In [31]:
# Calcul des métriques de classification
recall = recall_score(test_labels, y_pred, average='weighted')
f1 = f1_score(test_labels, y_pred, average='weighted')
accuracy = accuracy_score(test_labels, y_pred)

# Affichage des résultats
print("Recall score : {:.2f}%".format(recall*100))
print("F1-score : {:.2f}%".format(f1*100))
print("Accuracy score : {:.2f}%".format(accuracy*100))

Recall score : 71.43%
F1-score : 71.42%
Accuracy score : 71.43%


#Etape 5 : Conclusion

Les résultats obtenus indiquent que le modèle basé sur les LSTM est nettement supérieur au modèle basé sur les réseaux denses pour la tâche d'analyse de sentiments sur les tweets. En effet, les scores de précision, de rappel et de F1 sont tous supérieurs pour le modèle LSTM par rapport au modèle dense.

Cela peut s'expliquer par le fait que les réseaux de neurones récurrents de type LSTM sont particulièrement adaptés à la modélisation de séquences de données, comme les tweets. Les LSTM sont capables de mémoriser des informations à long terme, ce qui est particulièrement utile pour l'analyse de sentiments car le contexte des mots précédents peut influencer le sentiment exprimé dans un tweet.

En revanche, le modèle basé sur les réseaux denses ne prend pas en compte le contexte et considère chaque mot de manière indépendante, ce qui peut conduire à une moins bonne performance.

En conclusion, pour la tâche d'analyse de sentiments sur des tweets, le modèle basé sur les LSTM est à privilégier. Cependant, il est important de noter que la performance d'un modèle dépend également de la qualité et de la taille du jeu de données utilisé pour l'entraînement.